In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os

In [2]:
# csv paths
trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samsara_Trips.csv"
new_trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samara_trips_2025.xlsx"
trip_data = pd.read_csv(trip_csv)
end_path = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\generated data\samsara_wait_times.xlsx"

In [3]:
%%time
# csv paths
trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samsara_Trips.csv"
# trip_data = pd.read_csv(trip_csv)
trip_data = pd.read_excel(new_trip_csv)
trip_data = trip_data[trip_data['startAddress_name'] != ""]
trip_data['wait_time'] = ""
# make a list of dsates and one of vins
date_list = trip_data['Date'].unique()
vin_list = trip_data['Vin Number'].unique()
df_list = []
for vin in vin_list:
    vin_df = trip_data[trip_data['Vin Number']==vin]
    for day in date_list:
        date_subset = vin_df[vin_df['Date'] == day]
        date_subset = date_subset.reset_index()
        # make sure that this is still in the right order to calculate
        date_subset = date_subset.sort_values(by='start_time')
        df_length = len(date_subset)
        if df_length > 1:
            for row in range(0,df_length-1):
                current = date_subset.iloc[row]
                next_row = date_subset.iloc[row+1]
                if current['endAddress_name'] == next_row['startAddress_name']:
                    wait_time = pd.to_datetime(next_row['start_time']) - pd.to_datetime(current['end_time'])
                    date_subset.loc[row, 'wait_time'] = wait_time.total_seconds() / 60
            df_list.append(date_subset)

full_df = pd.concat(df_list)  

no_blank_times = full_df[full_df['wait_time'] != ""]
no_blank_times = no_blank_times[no_blank_times['wait_time'] < 180]
no_blank_times = no_blank_times[no_blank_times['wait_time'] > 10]
pivot_table = pd.pivot_table(no_blank_times, values='wait_time', index='endAddress_name', aggfunc='mean')

CPU times: total: 43.7 s
Wall time: 45.6 s


In [5]:
# create excel for spot checks
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
# full_df.to_excel(writer, sheet_name="full_df")
pivot_table.to_excel(writer, sheet_name="Pivot")
writer.close()

In [6]:
full_df.columns

Index(['index', 'CalendarSKey', 'id', 'startMs', 'endMs', 'startLocation',
       'endLocation', 'distanceMeters', 'fuelConsumedMl', 'tollMeters',
       'driverId', 'codriverIds', 'startOdometer', 'endOdometer', 'assetIds',
       'startAddress_id', 'startAddress_name', 'startAddress_address',
       'endAddress_id', 'endAddress_name', 'endAddress_address',
       'startCoordinates_latitude', 'startCoordinates_longitude',
       'endCoordinates_latitude', 'endCoordinates_longitude', 'vehicle_id',
       'start_time', 'end_time', 'Minutes', 'Gallons', 'Date', 'Miles', 'MPG',
       'Hours', 'UT%', 'EquipmentID', 'Vin Number', 'HRRef', 'Tags',
       'wait_time'],
      dtype='object')

In [ ]:
# total_len = len(full_df)
# over_3 = len(no_blank_times[no_blank_times['wait_time'] > 180])
# not_blank = len(no_blank_times)
# percent_wait = not_blank / total_len * 100
# percent_o3 = over_3 / not_blank *100

# print(f'there are {not_blank} rows out of {total_len}({percent_wait}%) and {percent_o3}% of those are over 3 hour waits')

In [28]:
%%time
# csv paths
trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samsara_Trips.csv"
# trip_data = pd.read_csv(trip_csv)
trip_data = pd.read_excel(new_trip_csv)
trip_data['wait_time'] = ""
# make a list of dsates and one of vins
date_list = trip_data['Date'].unique()
# date_list = ['01/02/2025']
# vin_list = ['1GT19LE79PF224344']
vin_list = trip_data['Vin Number'].unique()
# df_list = []

vin_subsets = [trip_data[trip_data['Vin Number'] == vin] for vin in vin_list] 
date_vin_subset = [x[x['Date'] == date]
                   for x in vin_subsets for date in date_list]


# date_subset = 

CPU times: total: 22.2 s
Wall time: 22.8 s


In [25]:
print(len(vin_subsets))

143
